# Reducing Failure-Inducing Inputs

_Brief abstract/introduction/motivation.  State what the chapter is about in 1-2 paragraphs._

**Prerequisites**

* _Refer to earlier chapters as notebooks here, as here:_ [Earlier Chapter](Fuzzer.ipynb).

## Why Reducing?

\todo{Add}

In [ ]:
import fuzzingbook_utils

In [ ]:
from Fuzzer import Runner

## Text-Based Input Reduction

\todo{Add}

In [ ]:
class Reducer(object):
    def __init__(self, runner, log=False):
        """Attach reducer to the given `runner`"""
        self.runner = runner
        self.log = log
        self.reset()

    def reset(self):
        self.tests = 0

    def test(self, inp):
        result, outcome = self.runner.run(inp)
        self.tests += 1
        if self.log:
            print("Test #%d" % self.tests, repr(inp), repr(len(inp)), outcome)
        return outcome

    def reduce(self, inp):
        self.reset()
        # Default: Don't reduce
        return inp

In [ ]:
class DeltaDebuggingReducer(Reducer):
    def reduce(self, inp):
        self.reset()
        assert self.test(inp) == Runner.FAIL

        n = 2     # Initial granularity
        while len(inp) >= 2:
            start = 0
            subset_length = len(inp) // n   # Integer Division
            some_complement_is_failing = False

            while start < len(inp):
                complement = inp[:start] + inp[start + subset_length:]

                if self.test(complement) == Runner.FAIL:
                    inp = complement
                    n = max(n - 1, 2)
                    some_complement_is_failing = True
                    break

                start += subset_length

            if not some_complement_is_failing:
                if n == len(inp):
                    break
                n = min(n * 2, len(inp))

        return inp

In [ ]:
class ParenRunner(Runner):
    def run(self, inp):
        if '(' in inp and ')' in inp:
            return (inp, self.FAIL)
        else:
            return (inp, self.PASS)

In [ ]:
paren_runner = ParenRunner()
dd_reducer = DeltaDebuggingReducer(paren_runner, log=True)
dd_reducer.reduce('foo(2 + 24)')

## Grammar-Based Input Reduction

\todo{Add}



In [ ]:
class GrammarReducer(Reducer):
    pass

## _Section 4_

\todo{Add}

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

The delta debugging algorithm discussed here stems from \cite{Zeller2002}; actually, this is the exact Python implementation as used by Zeller in 2002.  The idea of systematically reducing inputs has been discovered a number of times, although not as  automatic and generic as delta debugging. \cite{Slutz1998}, for instance, discusses systematic reduction of SQL statements for SQL databases; the general process as manual work is well described by \cite{Kernighan1999}.

\todo{Add more: hierarchical DD, grammar-based reduction, Csmith / C-reduce}


## Exercises

Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with

```markdown
**Solution.**
```

Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`).

Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution.

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_